![Yantrajaal Banner](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Postgres Exercises
Tutorial based on [SQLCourse2](https://www.sqlcourse2.com/) <br>


In [3]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2024-02-20 18:20:38.907668+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


#Install & Configure Postgres

In [4]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#!pip install psycopg2
import psycopg2
import pandas as pd
#!sudo -u postgres psql -V
!psql -V
!service postgresql start





psql (PostgreSQL) 14.10 (Ubuntu 14.10-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


#Create Schema, Tables and Insert Data

In [5]:
# This installation creates an Ubuntu user 'postgres' AND a Postgres database user 'postgres' both of which are superusers
#!cat /etc/passwd
# Use these users to create a new database user called 'upraxis' and give it a password 'upass'
#!sudo -u postgres createuser --superuser upraxis
!sudo -u postgres createuser upraxis
!sudo -u postgres psql -U postgres -c "ALTER USER upraxis PASSWORD 'upass';"
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
# Use ubuntu superuser to create database dbpraxis and set its owner to upraxis
!sudo -u postgres createdb -O upraxis dbpraxis
# Create file to store the password. Else will be prompted each time
!echo "localhost:5432:dbpraxis:postgres:pass" > ~/.pgpass
!echo "localhost:5432:dbpraxis:upraxis:upass" >> ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass


ALTER ROLE
ALTER ROLE
localhost:5432:dbpraxis:postgres:pass
localhost:5432:dbpraxis:upraxis:upass


In [6]:
# Download two CSV files from G-Drive into the Colab VM

#!gdown -qq 1Ug15GbEmeTnwJnuH-4fpPeZ1k20ryoGx      # Quiet Mode, not output
#!gdown -qq 1qx6duSqzQUJSkpHNQIIZ9l0T6zYtjT-y      # Quiet Mode, not output

!wget -O Customers.csv -q https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SQLCourse2%20-%20Customers.csv
!wget -O ItemsOrdered.csv -q https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SQLCourse2%20-%20ItemsOrdered.csv

In [7]:
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'drop table if exists itemsordered'
# --------------------------------------------------------------
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\
CREATE TABLE IF NOT EXISTS itemsordered (\
customerid char(5),orderdate date, item varchar(30), quantity int, price decimal(6,2)) \
'
# --------------------------------------------------------------
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'drop table if exists customers'
# --------------------------------------------------------------
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\
CREATE TABLE IF NOT EXISTS customers (customerid char(5),firstname varchar(20),lastname varchar(20), city varchar(20), state varchar(20)) \
'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\dt+'

NOTICE:  table "itemsordered" does not exist, skipping
DROP TABLE
CREATE TABLE
NOTICE:  table "customers" does not exist, skipping
DROP TABLE
CREATE TABLE
                                       List of relations
 Schema |     Name     | Type  |  Owner  | Persistence | Access method |  Size   | Description 
--------+--------------+-------+---------+-------------+---------------+---------+-------------
 public | customers    | table | upraxis | permanent   | heap          | 0 bytes | 
 public | itemsordered | table | upraxis | permanent   | heap          | 0 bytes | 
(2 rows)



In [8]:
# Only superuser ('postgres') can use copy command, not even database owner ('upraxis')
# 'postgres' password has to be entered manually, password = 'pass'
#
!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "COPY customers FROM '/content/Customers.csv' DELIMITER ',' CSV HEADER;"
!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "COPY itemsordered FROM '/content/ItemsOrdered.csv' DELIMITER ',' CSV HEADER;"

COPY 17
COPY 32


In [9]:
#!cat ItemsOrdered.csv
#!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'select * from itemsordered'

## Python Access
https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm

In [10]:
# Local Colab server

conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
#Creating a cursor object using the cursor() method
cursor = conn.cursor()
#Executing an MYSQL function using the execute() method
cursor.execute("select version()")
# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)
#Closing the connection
conn.close()

Connection established to:  ('PostgreSQL 14.10 (Ubuntu 14.10-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [11]:
# function to execute SQL select statements with Postgres
#
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return
# function to execute SQL non-select statements with Postgres
#
def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="exercises", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

##Check Data

In [12]:
#runSelect("select * from itemsordered")
runSelect("select * from customers")

customerid,firstname,lastname,city,state
10101,John,Gray,Lynden,Washington
10298,Leroy,Brown,Pinetop,Arizona
10299,Elroy,Keller,Snoqualmie,Washington
10315,Lisa,Jones,Oshkosh,Wisconsin
10325,Ginger,Schultz,Pocatello,Idaho
10329,Kelly,Mendoza,Kailua,Hawaii
10330,Shawn,Dalton,Cannon Beach,Oregon
10338,Michael,Howell,Tillamook,Oregon
10339,Anthony,Sanchez,Winslow,Arizona
10408,Elroy,Cleaver,Globe,Arizona


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)

In [13]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Last tested   2024-02-20 18:21:10.743463+05:30


#SELECT STATEMENT

In [14]:
#From the items_ordered table, select a list of all items purchased for customerid 10449. Display the customerid, item, and price for this customer.
runSelect("select customerid, item, price \
from itemsordered \
where customerid = '10449'")

customerid,item,price
10449,Unicycle,180.79
10449,Snow Shoes,45.00
10449,Bicycle,380.50
10449,Canoe,280.00
10449,Flashlight,4.50
10449,Canoe paddle,40.00


In [15]:
#Select all columns from the items_ordered table for whoever purchased a Tent
runSelect("select from itemsordered where item ='Tent'")

In [16]:
#Select the customerid, order_date, and item values from the items_ordered table for any items in the item column that start with the letter “S”.
runSelect("select * from itemsordered")

customerid,orderdate,item,quantity,price
10330,1999-06-30,Pogo stick,1,28.00
10101,1999-06-30,Raft,1,58.00
10298,1999-07-01,Skateboard,1,33.00
10101,1999-07-01,Life Vest,4,125.00
10299,1999-07-06,Parachute,1,1250.00
10339,1999-07-27,Umbrella,1,4.50
10449,1999-08-13,Unicycle,1,180.79
10439,1999-08-14,Ski Poles,2,25.50
10101,1999-08-18,Rain Coat,1,18.30
10449,1999-09-01,Snow Shoes,1,45.00


In [17]:
#2)Select the customerid, order_date, and item values from the items_ordered table for any items in the item column that start with the letter “S”.
runSelect("select customerid, orderdate, item, quantity, price \
from itemsordered \
where item = 'Tent'")

customerid,orderdate,item,quantity,price
10439,1999-09-18,Tent,1,88.00
10438,2000-01-18,Tent,1,79.99


In [18]:
#3)Select the customerid, order_date, and item values from the items_ordered table for any items in the item column that start with the letter “S”.
runSelect("select customerid, orderdate, item, quantity \
from itemsordered \
where item like 'S%'")

customerid,orderdate,item,quantity
10298,1999-07-01,Skateboard,1
10439,1999-08-14,Ski Poles,2
10449,1999-09-01,Snow Shoes,1
10410,1999-10-28,Sleeping Bag,1
10101,2000-03-08,Sleeping Bag,2
10330,2000-04-19,Shovel,1


In [19]:
#4)Select the distinct items in the items_ordered table. In other words, display a listing of each of the unique items from the items_ordered table.
runSelect("select distinct item \
from itemsordered \
")

item
Parachute
Canoe
Flashlight
Shovel
Umbrella
Ear Muffs
Inflatable Mattress
Ski Poles
Life Vest
Skateboard


In [20]:
#Make up your own select statements and submit them.
runSelect("select item, quantity \
from itemsordered \
where quantity > '1'")

item,quantity
Life Vest,4
Ski Poles,2
Lantern,2
Hoola Hoop,3
Flashlight,4
Lawnchair,4
Sleeping Bag,2
Canoe paddle,2


# AGGREGATE FUNCTION

In [21]:
#1)Select the maximum price of any item ordered in the items_ordered table. Hint: Select the maximum price only.
runSelect("select max(price) \
from itemsordered")

max
1250.00


In [22]:
#2)Select the average price of all of the items ordered that were purchased in the month of Dec
sql = "select orderdate, to_char(orderdate, 'YYYY/MM'), to_char(orderdate, 'MM'), price \
from itemsordered where to_char(orderdate, 'MM') = '12' "
runSelect(sql)

orderdate,to_char,to_char,price
1999-12-01,1999/12,12,22.00
1999-12-15,1999/12,12,380.50
1999-12-22,1999/12,12,280.00
1999-12-30,1999/12,12,14.75


In [23]:
runSelect("select avg(price) from itemsordered where to_char(orderdate, 'MM') = '12'")

avg
174.3125000000000000


In [24]:
#What are the total number of rows in the items_ordered tables
runSelect("select count(*) \
from itemsordered")

count
32


In [25]:
#For all of the tents that were ordered in the items_ordered table, what is the price of the lowest tent? Hint: Your query should return the price only.
runSelect("select min(price) \
from itemsordered")

min
4.50


# GROUP BY

In [26]:
#How many people are in each unique state in the customers table? Select the state and display the number of people in each. Hint: count is used to count rows in a column, sum works on numeric data only.
runSelect("select state, count(state) \
from customers \
group by state")

state,count
South Carolina,1
Washington,2
Oregon,2
Idaho,1
Colorado,2
Hawaii,1
Arizona,6
North Carolina,1
Wisconsin,1


In [27]:
#From the items_ordered table, select the item, maximum price, and minimum price for each specific item in the table. Hint: The items will need to be broken up into separate groups.
runSelect("select item, min(price), max(price) \
from itemsordered \
group by item")

item,min,max
Parachute,1250.00,1250.00
Canoe,280.00,280.00
Flashlight,4.50,28.00
Shovel,16.75,16.75
Umbrella,4.50,6.75
Ear Muffs,12.50,12.50
Inflatable Mattress,38.00,38.00
Ski Poles,25.50,25.50
Life Vest,125.00,125.00
Skateboard,33.00,33.00


In [32]:
#How many orders did each customer make? Use the items_ordered table. Select the customerid, number of orders they made, and the sum of their orders. Click the Group By answers link below if you have any problems.
runSelect("select customerid, count(customerid), sum(price) \
from itemsordered \
group by customerid")

customerid,count,sum
10299,2,1288.00
10410,2,281.72
10339,1,4.50
10298,5,118.88
10330,3,72.75
10438,3,95.24
10413,1,32.00
10439,2,113.50
10101,6,320.75
10449,6,930.79


In [37]:
#How many people are in each unique state in the customers table that have more than one person in the state? Select the state and display the number of how many people are in each if it’s greater than 1.
runSelect("select customerid, state \
from customers \
group by customerid \
having count(state) > '1'")

GroupingError: column "customers.state" must appear in the GROUP BY clause or be used in an aggregate function
LINE 1: select customerid, state from customers group by customerid ...
                           ^
